In [11]:
import pandas as pd
from RAKE.rake import Rake
import json
import ast
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import json
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/omai.r/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
matched_dfs = pd.read_csv('data/intermediate/matched_dfs.csv', converters = {'clean_text': ast.literal_eval})

In [4]:
#matched_dfs = pd.read_csv('data/intermediate/matched_dfs_merged.csv')

In [5]:
nature2references   = pd.read_hdf('data/nature/preprocessed/Nature2References.hdf')
nature_references_info = pd.read_hdf('data/nature/preprocessed/NatureReferencesInformation.hdf').set_index('DOI')

In [6]:
nature2references = pd.merge(nature2references, nature_references_info[['ArticleID']].reset_index(), right_on = 'ArticleID', left_on = 'CitingArticleID')

In [7]:
#matched_dfs = pd.merge(nature2references, matched_dfs,
#                       right_on = 'ArticleID', left_on = 'CitedArticleID')

matched_dfs = pd.merge(matched_dfs, nature_references_info[['ArticleID', 'Title']])

In [6]:
matched_dfs

,ArticleID,text,clean_text,count,Title
0,pub.1000000277,[['A detailed comparison of residues involved ...,[A detailed comparison of residues involved in...,1,Molecular Determinants of Agonist Discriminati...
1,pub.1000000280,"[['Also, the epigenetic stability of human ES ...","[Also, the epigenetic stability of human ES ce...",1,Temporal and parental-specific expression of i...
2,pub.1000000588,[['Hospital and clinic HTC had similar costs (...,[Hospital and clinic HTC had similar costs (&d...,1,What Is the Cost of Providing Outpatient HIV C...
3,pub.1000000718,"[['<b>e</b>, Allele-based pleiotropy of <i>IL6...","[<b>e</b>, Allele-based pleiotropy of <i>IL6R<...",2,Identification of multiple risk variants for a...
4,pub.1000000866,"[['However, structural<bibr rid=""b39 b40 b41 b...","[However, structural<bibr rid=""b39 b40 b41 b42...",1,"Crystal Structure of an α1,4-N-Acetylhexosamin..."
...,...,...,...,...,...
304443,pub.1121819253,[['Lineage-affiliated transcription factors<bi...,[Lineage-affiliated transcription factors<bibr...,4,GATA-1 interacts with the myeloid PU.1 transcr...
304444,pub.1121819356,[['\r\n<p>In order to investigate a yolk-sac c...,[\r\n<p>In order to investigate a yolk-sac con...,1,VEGF-C signaling pathways through VEGFR-2 and ...
304445,pub.1121824901,[['Despite the clear upper limit in observed a...,[Despite the clear upper limit in observed axi...,1,Sea-Floor Tectonics and Submarine Hydrothermal...
304446,pub.1122267945,[['The so-called photon-dominated or photodiss...,[The so-called photon-dominated or photodissoc...,7,Physics of the Interstellar and Intergalactic ...


In [7]:
matched_dfs.head()

,ArticleID,text,clean_text,count,Title
0,pub.1000000277,[['A detailed comparison of residues involved ...,[A detailed comparison of residues involved in...,1,Molecular Determinants of Agonist Discriminati...
1,pub.1000000280,"[['Also, the epigenetic stability of human ES ...","[Also, the epigenetic stability of human ES ce...",1,Temporal and parental-specific expression of i...
2,pub.1000000588,[['Hospital and clinic HTC had similar costs (...,[Hospital and clinic HTC had similar costs (&d...,1,What Is the Cost of Providing Outpatient HIV C...
3,pub.1000000718,"[['<b>e</b>, Allele-based pleiotropy of <i>IL6...","[<b>e</b>, Allele-based pleiotropy of <i>IL6R<...",2,Identification of multiple risk variants for a...
4,pub.1000000866,"[['However, structural<bibr rid=""b39 b40 b41 b...","[However, structural<bibr rid=""b39 b40 b41 b42...",1,"Crystal Structure of an α1,4-N-Acetylhexosamin..."


In [55]:
nature_references_info.head()

,ArticleID,PubYear,Title,Volume,Issue,Cites,Journal,NatureID
DOI,,,,,,,,
10.1038/166444b0,pub.1007899685,1950,Detection of Sugars on Paper Chromatograms,166,4219,3143,Nature,166444b0
10.1038/166117b0,pub.1026639700,1950,The Soil-Plate Method for Isolation of Fungi f...,166,4211,394,Nature,166117b0
10.1038/165514a0,pub.1039717355,1950,Reversible Dilation and Contraction by Changin...,165,4196,359,Nature,165514a0
10.1038/166780a0,pub.1011096023,1950,Cell Specificity of Histones,166,4227,280,Nature,166780a0
10.1038/166288a0,pub.1000041177,1950,Synthesis of Peptides in Enzymic Reactions inv...,166,4216,259,Nature,166288a0


In [8]:
matched_dfs.loc[4]['clean_text']

['However, structural and biochemical studies of several retaining glycosyltransferases in the presence of substrate analogues failed to identify an active-site nucleophile that could be involved in forming the enzyme–sugar adduct.']

In [23]:
rake = Rake("RAKE/SmartStoplist.txt")

### Test on a few papers

In [20]:
def top_trigram(text):
    #print(text)
    
    
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(' '.join(text))
    filtered_sentence = [w for w in word_tokens if (w not in stop_words) and (w.isalpha())]
    
    distribution = nltk.FreqDist(nltk.trigrams(filtered_sentence))
    distribution =  pd.DataFrame.from_dict(distribution, orient = 'index')
    distribution['freq'] = distribution[0] / distribution[0].sum()
    
    
    
    return distribution.sort_values('freq', ascending = False).iloc[:10]

In [23]:
matched_dfs.iloc[45845]['clean_text']

['Subjects were assigned a location in a network of 20 nodes, generated by a preferential attachment model; the network structure was created de novo for each session by attaching new nodes (each with two links) to existing nodes; and subjects were placed into the resulting networks at random.',
 'A soft thresholding power was chosen on the basis of the criterion of approximate scale-free topology.',
 'ÁngelesSerranoM\nMariánBoguñáM\nDmitriKrioukovD\nDepartment of Electrical Engineering, Computer Engineering and Informatics, Cyprus University of Technology, 33 Saripolou Street, 3036 Limassol, Cyprus\nCooperative Association for Internet Data Analysis (CAIDA), University of California, San Diego (UCSD), La Jolla, California 92093, USA\nDepartament de Física Fonamental, Universitat de Barcelona, Martí i Franquès 1, 08028 Barcelona, Spain\nf.papadopoulos@cut.ac.cy\ndima@ucsd.edu\nA framework is developed in which new connections to a growing network optimize geometric trade-offs between p

In [24]:
top_trigram(matched_dfs.iloc[45845]['clean_text'])

,0,freq
"(networks, degree, distributions)",2,0.003317
"(highly, connected, proteins)",2,0.003317
"(networks, social, networks)",2,0.003317
"(random, graphs, networks)",2,0.003317
"(protein, interaction, networks)",2,0.003317
"(degree, distribution, Here)",2,0.003317
"(Internet, Data, Analysis)",1,0.001658
"(attachment, This, implies)",1,0.001658
"(PB, k, lgr)",1,0.001658
"(spatial, lattices, random)",1,0.001658


In [86]:
test_df = matched_dfs.sort_values('count', ascending = False).iloc[:300].sample(5)

In [87]:
for index, paper in test_df.iterrows():
    
    print('\n\n')
    print('Paper Title: %s \n\nPaper DOI: %s \n\n Paper Keywords: %s\n\n' % (paper['Title'],paper['DOI'],
                                                                          str(rake.run(paper['clean_text']))))
    




Paper Title: A Bivalent Chromatin Structure Marks Key Developmental Genes in Embryonic Stem Cells 

Paper Keywords: 10.1016/j.cell.2006.02.041 

 PaperID: [('primary human fibroblast cell types', 21.5), ('actively silence differentiation-affiliated transcription factors', 20.7), ('h3k27me pathway disrupt cardiac development', 19.3), ('chromatin regulatory networks controlling pluripotency', 18.77894736842105), ('bivalent domains mark developmentally important loci', 17.57016632016632), ('systematically analysed chromatin-modifying proteins', 15.0), ('non-expressed target genes adopts', 14.45945945945946), ('maps show close agreement', 13.5), ('mouse cage tag clusters', 13.4), ('previously reported chip-chip data', 13.0), ('properly restore bivalent domains', 12.793243243243243), ('human frontal cortex h3k4me3', 12.666666666666666), ('regulating key developmental genes', 12.316602316602317), ('poised h3k4me3+h3k27me3+ bivalent status', 11.925061425061426), ('mark active enhancers spe

In [91]:
test_df[test_df['DOI'] == '10.1016/j.cell.2006.02.041']['clean_text'].values

array(['[\'The 5′ element, for which we suggest the name HOTTIP for ‘HOXA transcript at the distal tip’, exhibits bivalent H3K4me3 and H3K27me3, a histone modification pattern associated with poised regulatory sequences.\', \'The combination of both activating (H3K4 methylation) and inactivating (H3K27 methylation) chromatin marks characterizes ‘poised’ promoters and enhancers, which regulate expression of key developmental genes.\', \'H3K4me is an activating mark found in promoters/enhancers of key developmental genes.\', \'These findings establish that mutations in many genes in the H3K4me–H3K27me pathway disrupt cardiac development and are consistent with previous evidence implicating these chromatin marks in regulating key developmental genes, including those involved in cardiac development.\', \'Lysine 27 is a critical residue of histone 3 and its variants, and methylation at this position (H3K27me), which may be mimicked by the terminal CH3 of methionine substituted at this resid

### Investigating Keywords for Scale Free

In [9]:
matched_dfs[matched_dfs['Title'].str.contains(r'Scaling.*Random.*')]\
            .sort_values('count', ascending = False)

,ArticleID,text,clean_text,count,Title
45845,pub.1010080128,[['Subjects were assigned a location in a netw...,[Subjects were assigned a location in a networ...,31,Emergence of Scaling in Random Networks


In [18]:
#rake = Rake('RAKE/SmartStoplist.txt')
#rake.run(matched_dfs.iloc[45845]['clean_text'])

In [25]:
top_trigram(matched_dfs.iloc[45845]['clean_text'])

,0,freq
"(networks, degree, distributions)",2,0.003317
"(highly, connected, proteins)",2,0.003317
"(networks, social, networks)",2,0.003317
"(random, graphs, networks)",2,0.003317
"(protein, interaction, networks)",2,0.003317
"(degree, distribution, Here)",2,0.003317
"(Internet, Data, Analysis)",1,0.001658
"(attachment, This, implies)",1,0.001658
"(PB, k, lgr)",1,0.001658
"(spatial, lattices, random)",1,0.001658


In [42]:
matched_dfs[matched_dfs['Title'].str.contains(r'.*Coot:.*')]\
            .sort_values('count', ascending = False)
#[['Title', 'DOI', 'ArticleID', 'count', 'Cites']]

,ArticleID,text,clean_text,count,Title
92203,pub.1020281098,[['This model was combined into a single prote...,[This model was combined into a single protein...,520,Coot: model-building tools for molecular graphics


In [ ]:
#matched_dfs[matched_dfs['Title'].str.contains(r'.*Coot:.*')]\
#            .sort_values('count', ascending = False)
#[['Title', 'DOI', 'ArticleID', 'count', 'Cites']]

In [76]:
matched_dfs[matched_dfs['Title'].str.contains(r'.*MOLECULAR STRUCTURE OF NUCLEIC ACIDS.*', case = False)]\
            .sort_values('count', ascending = False)
#[['Title', 'DOI', 'ArticleID', 'count', 'Cites']]

,ArticleID,text,clean_text,count,Title
183344,pub.1040343773,"[['In addition, the design of nucleic acids th...","[In addition, the design of nucleic acids that...",20,Molecular Structure of Nucleic Acids: A Struct...
10716,pub.1002346037,[['Yet in 1953 <i>Nature</i> published seven p...,[Yet in 1953 Nature published seven papers on ...,2,Molecular Structure of Nucleic Acids: Molecula...


In [27]:
top_trigram(matched_dfs.iloc[183344]['clean_text'])

,0,freq
"(Watson, Francis, Crick)",3,0.007958
"(James, Watson, Francis)",3,0.007958
"(DNA, double, helix)",3,0.007958
"(immediately, suggests, possible)",2,0.005305
"(suggests, possible, copying)",2,0.005305
"(possible, copying, mechanism)",2,0.005305
"(copying, mechanism, genetic)",2,0.005305
"(DNA, Until, however)",1,0.002653
"(adenine, forms, hydrogen)",1,0.002653
"(seven, papers, structure)",1,0.002653


In [57]:
' '.join(matched_dfs.iloc[183344]['clean_text'])

"In addition, the design of nucleic acids that bind to other nucleic acids with high affinity and specificity follows the simple and long-established rules of Watson–Crick base pairing. The discovery of the double helix in 1953 immediately raised questions about how biological information is encoded in DNA. As implied by James Watson and Francis Crick in their landmark paper, base pairing suggests a template-copying mechanism that accounts for the fidelity in copying of genetic material during DNA replication (see article in this issue by Alberts, page 431). A DNA duplex for redundancy\nWatson and Crick noted, with infamous prophetic understatement, “it has not escaped our notice that the specific [base] pairing we have postulated immediately suggests a possible copying mechanism for the genetic material”. Following the discovery of DNA as the molecular substrate of inheritance and determination of its structure, this speculation was supported by the demonstration that agents that dama

## RAKE

In [61]:
rake.run(' '.join(matched_dfs.iloc[92203]['clean_text']))

[('experimentally phased ncs averaged electron density map permitted',
  32.08662587412587),
 ('protein data bank accession number 4r12', 26.850215432824125),
 ('protein data bank accession number 4hyc', 26.850215432824125),
 ('protein data bank accession number 3mku', 26.850215432824125),
 ('resulting maps showed clear unbiased density', 24.68044871794872),
 ('selenium anomalous difference fourier peak', 22.0),
 ('highly reliable initial rigid body fit', 21.57804459691252),
 ('multi-domain averaged electron density maps', 20.925641025641024),
 ('iterative process involving careful model building', 20.778171702225674),
 ('two-fold symmetry averaging produced experimental maps',
  19.948484848484846),
 ('2\u2009å resolution sigmaa-weighted difference fourier maps',
  18.833333333333332),
 ('composite annealed omit maps calculated', 18.409523809523808),
 ('coot applying ten-fold ncs operations', 18.004032258064516),
 ('resulting phase-combined electron density map', 16.836625874125872),


In [40]:
top_trigram(matched_dfs.iloc[92203]['clean_text'])


,0,freq
"(The, model, built)",25,0.004197
"(manual, model, building)",23,0.003861
"(model, built, using)",19,0.003190
"(model, building, refinement)",18,0.003022
"(performed, using, COOT)",15,0.002518
"(Model, building, performed)",15,0.002518
"(model, building, COOT)",15,0.002518
"(initial, model, built)",15,0.002518
"(building, refinement, performed)",15,0.002518
"(Model, building, refinement)",12,0.002014


In [104]:
test_trigrams = top_trigram(matched_dfs.iloc[92208]['clean_text'])

### Build TF-IDF based key words system

In [28]:
#clean_text_corpus = [' '.join(ast.literal_eval(i)) for i in tqdm(matched_dfs['clean_text'].tolist())]
clean_text_corpus = [' '.join(i) for i in tqdm(matched_dfs['clean_text'].tolist())]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [9]:
clean_text_corpus[:50]

['A detailed comparison of residues involved in glycine binding to NR1 subunits<bibr rid="b8 b9 b10"/> and glutamate binding to NR2 subunits<bibr rid="b11 b12"/> revealed that NR3B (or NR3A) could potentially share ligand specificity with either or both of these classes of NMDAR subunit.',
 'Also, the epigenetic stability of human ES cells seems to be greater than mouse ES cells, as shown here and previously by their extent of monoallelic imprinted gene expression<bibr rid="b23 b24"/>.',
 'Hospital and clinic HTC had similar costs (&dollar;12.56 and &dollar;12.32, respectively)<bibr rid="b81 b88 b93 b140 b142 b143 b144 b145 b146 b147"/> (Supplementary Table S3 and Figure S62).',
 '<b>e</b>, Allele-based pleiotropy of <i>IL6R</i> 358Asp (rs2228145 (A))<bibr rid="b5"/> on multiple disease phenotypes, including increased risk of RA, ankylosing spondylitis and coronary heart disease (asterisks indicate associations obtained from the literature<bibr rid="b29 b30"/>) and protection from asth

In [29]:
def tf_idfer(clean_text_corpus):

    print('Getting stop words.. ')
    stop_words = set(stopwords.words('english')) 

    print('Tokenizing the sentences..')
    clean_text_corpus = [word_tokenize(sent) for sent in clean_text_corpus]

    print('Cleaning the sentences..')
    cleaner = lambda sent: [w for w in sent if (w not in stop_words) and (w.isalpha())]
    clean_text_corpus = list(map(cleaner, clean_text_corpus))

    print('Joining the words back to make a sentence..')
    clean_text_corpus = [' '.join(i) for i in clean_text_corpus]

    print('Vectorizing the sentence..')
    vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 3))
    filtered_corpus_vector = vectorizer.fit_transform(clean_text_corpus)
    
    #print(len(vectorizer.get_feature_names()))
    #print(filtered_corpus_vector.toarray())
    
    return vectorizer, filtered_corpus_vector
    

In [30]:
%%time
vectorizer, filtered_corpus_vector = tf_idfer(clean_text_corpus)

Getting stop words.. 
Tokenizing the sentences..
Cleaning the sentences..
Joining the words back to make a sentence..
Vectorizing the sentence..
CPU times: user 5min 3s, sys: 3.74 s, total: 5min 7s
Wall time: 5min 7s


##### The one with no mapping has a lot of sentences. This should be removed

In [31]:
[len(i) for i in clean_text_corpus[:10]]

[242, 176, 130, 654, 229, 284, 183, 147, 146, 302]

In [32]:
trigrams = vectorizer.get_feature_names()

In [33]:
transformer = TfidfTransformer(smooth_idf=True)

In [34]:
tf_idf = transformer.fit_transform(filtered_corpus_vector)

In [35]:
#tf_idf = tf_idf[1:]
#filtered_corpus_vector = filtered_corpus_vector[1:]

In [36]:
filtered_corpus_vector.shape

(304448, 8566042)

In [37]:
tf_idf.shape

(304448, 8566042)

In [38]:
len(matched_dfs)

304448

### Looking into the tf-idf for coot to see if it makes sense.

In [47]:
matched_dfs.loc[92203]

ArticleID                                        pub.1020281098
text          [['This model was combined into a single prote...
clean_text    [This model was combined into a single protein...
count                                                       520
Title         Coot: model-building tools for molecular graphics
Name: 92203, dtype: object

In [48]:
trigrams[np.argmax(tf_idf[92203])]

'using coot'

In [51]:
matched_dfs.loc[92203]['Title']

'Coot: model-building tools for molecular graphics'

In [52]:
for i in sorted(np.argsort(tf_idf[92203].todense())[:5].tolist()[0][-10:], reverse = True):
    print(trigrams[i])

using coot
the model
program coot
model built
model building
coot the
coot refinement
coot refined
coot phenix
building coot


#### Woo! It makes sense! Trying on other papers

In [74]:
test_df = matched_dfs.sort_values('count', ascending = False).iloc[:300].sample(5)

for index, paper in test_df.iterrows():
    
    print('\n\n')
    print('Paper %s ' % paper['Title'])
    print('Key Phrases:\n')
    for i in sorted(np.argsort(tf_idf[index].todense())[:5].tolist()[0][-10:], reverse = True):
        print(trigrams[i])




Paper Quantum phase transition from a superfluid to a Mott insulator in a gas of ultracold atoms 
Key Phrases:

transition superfluid
quantum phase transition
quantum phase
phase transition
optical lattice
mott insulator
mott insulating state
mott insulating
lattice potential
insulating state



Paper Sequence and comparative analysis of the chicken genome provide unique perspectives on vertebrate evolution 
Key Phrases:

ucd line
red jungle fowl
red jungle
partly inbred ucd
partly inbred
jungle fowl
inbred ucd line
inbred ucd
female partly inbred
chicken genome



Paper Role of Tet proteins in 5mC to 5hmC conversion, ES-cell self-renewal and inner cell mass specification 
Key Phrases:

tet proteins
tet family
stem es cells
stem es
mouse es cells
mouse es
massachusetts usa
es cells
embryonic stem es
embryonic stem



Paper Structure of the TRPV1 ion channel determined by electron cryo-microscopy 
Key Phrases:

trp channels
trp channel structures
previously described
in vgics
distinc

In [189]:
matched_dfs.iloc[45845]['clean_text']

['Subjects were assigned a location in a network of 20 nodes, generated by a preferential attachment model; the network structure was created de novo for each session by attaching new nodes (each with two links) to existing nodes; and subjects were placed into the resulting networks at random.',
 'A soft thresholding power was chosen on the basis of the criterion of approximate scale-free topology.',
 'ÁngelesSerranoM\nMariánBoguñáM\nDmitriKrioukovD\nDepartment of Electrical Engineering, Computer Engineering and Informatics, Cyprus University of Technology, 33 Saripolou Street, 3036 Limassol, Cyprus\nCooperative Association for Internet Data Analysis (CAIDA), University of California, San Diego (UCSD), La Jolla, California 92093, USA\nDepartament de Física Fonamental, Universitat de Barcelona, Martí i Franquès 1, 08028 Barcelona, Spain\nf.papadopoulos@cut.ac.cy\ndima@ucsd.edu\nA framework is developed in which new connections to a growing network optimize geometric trade-offs between p